In [1]:
from transformers.models.bart.modeling_bart import BartForConditionalGeneration, BartEncoder, BartConfig, BartDecoder, shift_tokens_right
from transformers import BartTokenizer

2023-02-21 17:18:49.110812: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-21 17:18:49.467757: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-21 17:18:49.467921: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-21 17:18:51.568982: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [9]:
# tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
# model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
	
tokenizer = BartTokenizer.from_pretrained('sshleifer/distilbart-xsum-6-6')
model = BartForConditionalGeneration.from_pretrained('sshleifer/distilbart-xsum-6-6')

In [11]:
tokenized = tokenizer('Penguins, unlike most birds, they can\'t', return_tensors='pt')
output = model.generate(**tokenized)
decoded = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded)

/home/bary/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 62 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Penguins have been released into the wild for the first time.


In [48]:
import math
import copy
import torch
import torch.nn as nn
from typing import Optional, List, Tuple, Union
from transformers.models.bart.modeling_bart import BartEncoderLayer, BartDecoderLayer, BartConfig


class MyDecoderLayer(nn.Module):
    def __init__(self, layer: BartDecoderLayer, config: BartConfig):
        super().__init__()
        self.embed_dim = layer.embed_dim
        self.self_attn = layer.self_attn
        self.dropout = layer.dropout

        self.activation_fn = layer.activation_fn
        self.activation_dropout = layer.activation_dropout

        self.self_attn_layer_norm = layer.self_attn_layer_norm
        self.encoder_attn = layer.encoder_attn
        self.num_encoder_attns = 2
        self.encoder_attns = nn.ModuleList([copy.deepcopy(layer.encoder_attn) for _ in range(self.num_encoder_attns)])
        self.encoder_attn_layer_norm = layer.encoder_attn_layer_norm
        self.fc1 = layer.fc1
        self.fc2 = layer.fc2
        self.final_layer_norm = layer.final_layer_norm

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        encoder_hidden_states: Optional[torch.Tensor] = None,
        encoder_attention_mask: Optional[torch.Tensor] = None,
        layer_head_mask: Optional[torch.Tensor] = None,
        cross_attn_layer_head_mask: Optional[torch.Tensor] = None,
        past_key_value: Optional[Tuple[torch.Tensor]] = None,
        output_attentions: Optional[bool] = False,
        use_cache: Optional[bool] = True,
    ) -> Tuple[torch.FloatTensor, Optional[Tuple[torch.FloatTensor, torch.FloatTensor]]]:
        """
        Args:
            hidden_states (`torch.FloatTensor`): input to the layer of shape `(batch, seq_len, embed_dim)`
            attention_mask (`torch.FloatTensor`): attention mask of size
                `(batch, 1, tgt_len, src_len)` where padding elements are indicated by very large negative values.
            encoder_hidden_states (`torch.FloatTensor`):
                cross attention input to the layer of shape `(num_context_sequences, batch, seq_len, embed_dim)`
            encoder_attention_mask (`torch.FloatTensor`): encoder attention mask of size
                `(batch, 1, tgt_len, src_len)` where padding elements are indicated by very large negative values.
            layer_head_mask (`torch.FloatTensor`): mask for attention heads in a given layer of size
                `(encoder_attention_heads,)`.
            cross_attn_layer_head_mask (`torch.FloatTensor`): mask for cross-attention heads in a given layer of
                size `(decoder_attention_heads,)`.
            past_key_value (`Tuple(torch.FloatTensor)`): cached past key and value projection states
            output_attentions (`bool`, *optional*):
                Whether or not to return the attentions tensors of all attention layers. See `attentions` under
                returned tensors for more detail.
        """
        residual = hidden_states

        # Self Attention
        # decoder uni-directional self-attention cached key/values tuple is at positions 1,2
        self_attn_past_key_value = past_key_value[:2] if past_key_value is not None else None
        # add present self-attn cache to positions 1,2 of present_key_value tuple
        hidden_states, self_attn_weights, present_key_value = self.self_attn(
            hidden_states=hidden_states,
            past_key_value=self_attn_past_key_value,
            attention_mask=attention_mask,
            layer_head_mask=layer_head_mask,
            output_attentions=output_attentions,
        )
        hidden_states = nn.functional.dropout(hidden_states, p=self.dropout, training=self.training)
        hidden_states = residual + hidden_states
        hidden_states = self.self_attn_layer_norm(hidden_states)

        # Cross-Attention Block
        cross_attn_present_key_value = None
        cross_attn_weights = None
        if encoder_hidden_states is not None:
            residual = hidden_states

            # cross_attn cached key/values tuple is at positions 3,4 of present_key_value tuple
            cross_attn_past_key_value = past_key_value[-2:] if past_key_value is not None else None
            hidden_states_all = []
            cross_attn_weights_all = []
            cross_attn_present_key_value = []
            for i in range(self.num_encoder_attns):
                hidden_states, cross_attn_weights, cross_attn_present_key_value = self.encoder_attn(
                    hidden_states=hidden_states,
                    key_value_states=encoder_hidden_states[i,:,:,:].squeeze(0),
                    attention_mask=encoder_attention_mask,
                    layer_head_mask=cross_attn_layer_head_mask,
                    past_key_value=cross_attn_past_key_value,
                    output_attentions=output_attentions,
                )
                hidden_states_all.append(hidden_states)
                cross_attn_weights_all.append(cross_attn_weights)
                cross_attn_present_key_value.append(cross_attn_present_key_value)

            hidden_states_all = torch.stack(hidden_states_all, dim=0)
            cross_attn_weights_all = torch.stack(cross_attn_weights_all, dim=0)
            cross_attn_present_key_value = torch.stack(cross_attn_present_key_value, dim=0)

            hidden_states = hidden_states_all.mean(dim=0)
            cross_attn_weights = cross_attn_weights_all.mean(dim=0)
            cross_attn_present_key_value = cross_attn_present_key_value.mean(dim=0)

            hidden_states = nn.functional.dropout(hidden_states, p=self.dropout, training=self.training)
            hidden_states = residual + hidden_states
            hidden_states = self.encoder_attn_layer_norm(hidden_states)

            # add cross-attn to positions 3,4 of present_key_value tuple
            present_key_value = present_key_value + cross_attn_present_key_value

        # Fully Connected
        residual = hidden_states
        hidden_states = self.activation_fn(self.fc1(hidden_states))
        hidden_states = nn.functional.dropout(hidden_states, p=self.activation_dropout, training=self.training)
        hidden_states = self.fc2(hidden_states)
        hidden_states = nn.functional.dropout(hidden_states, p=self.dropout, training=self.training)
        hidden_states = residual + hidden_states
        hidden_states = self.final_layer_norm(hidden_states)

        outputs = (hidden_states,)

        if output_attentions:
            outputs += (self_attn_weights, cross_attn_weights)

        if use_cache:
            outputs += (present_key_value,)

        return outputs

In [45]:


for i in range(len(model.model.decoder.layers)):
    layer = model.model.decoder.layers[i]
    new_layer = MyDecoderLayer(layer, model.model.config)
    model.model.decoder.layers[i] = new_layer

In [49]:
output1 = model.model.encoder(**tokenized, output_hidden_states=True, output_attentions=True)
output2 = model.model.encoder(**tokenized, output_hidden_states=True, output_attentions=True)

In [57]:
encoder_hidden_states = torch.stack((output1.last_hidden_state, output2.last_hidden_state))
decoder_input_ids = tokenizer.encode('', return_tensors='pt')

In [63]:
output = model.model.decoder(encoder_hidden_states=encoder_hidden_states, input_ids=decoder_input_ids)

In [71]:
model.lm_head(output.last_hidden_state) + model.final_logits_bias

tensor([[31.3633,  2.8388,  1.5540,  ...,  3.7043,  3.5683, -1.6295],
        [ 1.6151, -1.2754,  4.4009,  ..., -1.5303, -1.5515, -2.1655]],
       grad_fn=<AddBackward0>)

In [53]:
model.forward?

Signature:
model.forward(
    input_ids: torch.LongTensor = None,
    attention_mask: Optional[torch.Tensor] = None,
    decoder_input_ids: Optional[torch.LongTensor] = None,
    decoder_attention_mask: Optional[torch.LongTensor] = None,
    head_mask: Optional[torch.Tensor] = None,
    decoder_head_mask: Optional[torch.Tensor] = None,
    cross_attn_head_mask: Optional[torch.Tensor] = None,
    encoder_outputs: Optional[List[torch.FloatTensor]] = None,
    past_key_values: Optional[List[torch.FloatTensor]] = None,
    inputs_embeds: Optional[torch.FloatTensor] = None,
    decoder_inputs_embeds: Optional[torch.FloatTensor] = None,
    labels: Optional[torch.LongTensor] = None,
    use_cache: Optional[bool] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    return_dict: Optional[bool] = None,
) -> Union[Tuple, transformers.modeling_outputs.Seq2SeqLMOutput]
Docstring:
The [`BartForConditionalGeneration`] forward method, overrides th